In [2]:
import json
import pandas as pd

In [4]:
with open('newTweets.json') as json_file:
    data = json.load(json_file)
data = [d for d in data.values()]
new_tweets = pd.json_normalize(data)
new_tweets.head(1)

,text,created_at,id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count
0,"You are my favorite hello,\nAnd my hardest goo...",2022-03-07T07:49:42.000Z,1500740461115461632,6122,4693,44162,1108


In [5]:
kaggle_tweets = pd.read_csv("2021.csv")
kaggle_tweets.head(1)

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,0,1373819373090050048,1373669212271566858,1.616379e+12,2021-03-22 02:10:37,0,NaN,@bluemoondance74 @NASASpaceflight Going well. ...,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'bluemoondance74', 'name': 'R...",NaN,NaN,NaN,NaN


In [6]:
print(kaggle_tweets.columns, new_tweets.columns, sep="\n")

Index(['Unnamed: 0', 'id', 'conversation_id', 'created_at', 'date', 'timezone',
       'place', 'tweet', 'language', 'hashtags', 'cashtags', 'user_id',
       'user_id_str', 'username', 'name', 'day', 'hour', 'link', 'urls',
       'photos', 'video', 'thumbnail', 'retweet', 'nlikes', 'nreplies',
       'nretweets', 'quote_url', 'search', 'near', 'geo', 'source',
       'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date',
       'translate', 'trans_src', 'trans_dest'],
      dtype='object')
Index(['text', 'created_at', 'id', 'public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count',
       'public_metrics.quote_count'],
      dtype='object')


In [8]:
# Rename and remove columns to get same format
kaggle_tweets_map = {}
kaggle_tweets_drop = ['Unnamed: 0', 'conversation_id', 'date', 'timezone', 'place', 'language', 'hashtags', 'cashtags', 'user_id', 
                     'user_id_str', 'username', 'name', 'day', 'hour', 'link', 'urls', 
                     'photos', 'video', 'thumbnail', 'retweet', 
                     'quote_url', 'search', 'near', 'geo', 'source', 
                     'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date',
                     'translate', 'trans_src', 'trans_dest']

new_tweets_map = {'text': 'tweet',
                  'public_metrics.retweet_count': 'nretweets',
                  'public_metrics.reply_count': 'nreplies',
                  'public_metrics.like_count': 'nlikes'}
new_tweets_drop = ['public_metrics.quote_count']

edited_kaggle_tweets = kaggle_tweets.rename(columns=kaggle_tweets_map).drop(columns=kaggle_tweets_drop)
edited_new_tweets = new_tweets.rename(columns=new_tweets_map).drop(columns=new_tweets_drop)

# Remove the 6 duplicate tweets :
# (1373819373090050048, 1373735946244431873, 1373555480870621188, 1373507545315172357, 1373492611231535111, 1373490900807032840)
edited_new_tweets = edited_new_tweets[:-6]

# Remove \n in new tweets.
edited_new_tweets['tweet'].replace(to_replace=r'\n', value=' ', regex=True, inplace=True) 

# Get all date in same format.
edited_new_tweets['created_at'] = pd.to_datetime(edited_new_tweets['created_at'], infer_datetime_format=True).dt.tz_convert(None)
edited_kaggle_tweets['created_at'] = pd.to_datetime(edited_kaggle_tweets['created_at'],  unit='ms')

# Dump to CSV:
pd.concat([edited_new_tweets, edited_kaggle_tweets], ignore_index=True, verify_integrity=True)[['id', 'created_at', 'tweet', 'nlikes', 'nreplies', 'nretweets']].to_csv("alltweets.csv")